# 列挙型・共用体・バリアント型・オプショナル型


## キーポイント

* 列挙型は「複数の名前をグループ化する」機能
* 列挙型を使うと「互いに異なるものとして扱いたいが、数値でも文字列でもあらわしにくいデータ」を表現できる
* 列挙型は`enum class`キーワードを使って次のように定義する
    ```cpp
    enum class 列挙型の名前 {
      列挙型のメンバの名前(その1),
      列挙型のメンバの名前(その2),
      ︙
    };
    ```
* 共用体は「メモリの節約」を目的とした特殊なクラス
* 共用体では、同じメモリアドレスが複数のメンバ変数で共有され、一度にひとつのメンバ変数だけを有効にできる
* 共用体は`union`キーワードを使って次のように定義する
    ```cpp
    union 共用体の名前 {
      メンバ変数１の型  メンバ変数１の名前;
      メンバ変数２の型  メンバ変数２の名前;
      ︙
    };
    ```
* 共用体では、有効なメンバはプログラマが管理しなくてはならない
* `variant`(バリアント)型は「安全な共用体」で、有効なメンバの管理などが自動化されている
* C++では共用体より`variant`型を使うべき
* `optional`(オプショナル)型は「値が無効な状態」と「値が有効な状態」の2つの状態をあらわせる型
* `optional`型は「失敗し得る関数の戻り値」として使う


## 1 列挙型(れっきょがた)



### 1.1 列挙型を定義する

列挙型は「複数の名前をグループ化する」機能です。C++では、次の2種類の列挙型が使えます。

* `enum`(イナム、またはイーナム): Cの頃からある古い列挙型
* `enum class`(イナム・クラス):&emsp; C++11で追加された「より安全な」列挙型

基本的には、安全性の高い`enum class`が使われます。<br>
`enum`を使うのは、C++11以前のバージョンや、C言語との互換性が必要な場合くらいです。

>`enum`は`enumerate`(イナムレート、「数え上げる」「列挙する」という意味)の短縮形です。

列挙型を定義するには、`enum class`キーワードを使って次のように書きます。

```cpp
enum class 列挙型の名前 {
  列挙型のメンバの名前(その1),
  列挙型のメンバの名前(その2),
  ︙
};
```

メンバ同士は`,`(カンマ)記号で区切ります。<br>
例えば、トランプのカードの種類をあらわす列挙型は、次のようになります。

```cpp
//         ↓ 列挙型の名前
enum class Suit {
  Club,    // ← 列挙型のメンバ(列挙子)その1
  Diamond, // ← 列挙子その2(列挙型のメンバは「カンマ」記号で区切る
  Heart,   // ← 列挙子その3
  Spade,   // ← 列挙子その4
};
```

この例では、`Suit`(スート)という名前の列挙型を定義し、Club(クラブ)、Diamond(ダイヤモンド)、Heart(ハート)、Spade(スペード)という4つの名前を、メンバとしてグループ化しています。

グループのメンバは「列挙子(れっきょし)」と呼ばれ、「`static`かつ`const`なメンバ変数」のように扱われます。<br>
つまり、読むことはできても代入はできません。

列挙型と列挙子は次のように使います。

```cpp
Suit s; // Suit(スート)型の変数 s を宣言(値は不定)
s = Suit::Club; // 変数 s に列挙子 Suit::Club を代入

Suit t = Suit::Diamond; // Suit(スート)型の変数 t を宣言し、列挙子 Suit::Diamond で初期化
s = t; // 変数 s に変数 t を代入

// Suit::Heart = Suit::Club; // エラー. 列挙子への代入はできない
```

>列挙型は「名前をグループ化する」機能です。<br>
>構造体やクラスとは異なり、列挙型のメンバが変数ごとに作成されたりはしません。


### 1.2 列挙子の値

C++の列挙型では、個々の列挙子に整数が割り当てられます。割り当てのルールは次のとおりです。

* 列挙子に整数を割り当てるには`=`記号を使う
* 列挙子に整数を割り当てなかった場合、自動的に「すぐ上の列挙子の整数 + 1」が割り当てられる
* 整数を割り当てなかったのが「先頭の列挙子」だった場合、自動的に`0`が割り当てられる。

次のプログラムは、値の割り当てルールを使った例です。

```cpp
enum class Color {
  Black,    // 値の指定がなく、先頭の列挙子なので、0が割り当てられる
  Blue = 3, // =記号で3を指定しているので、Blueには3が割り当てられる
  Green,    // 値の指定がないので、ひとつ上の値に1を足した4が割り当てられる
  Red = -1,  // =記号で-1を指定しているので、Redには-1が割り当てられる
};
```

列挙型と整数の演算はできませんが、キャストを使えば整数型に変換できます。

```cpp
int a = 3; // OK.
// int b = Color::Blue; // エラー. 列挙子を直接代入することはできない
int c = (int)Color::Blue; // OK. 数値型にキャストすれば代入できる
```

この逆で、整数型から列挙型への変換も可能です。

```cpp
//Color a = 4; // エラー. 整数を列挙型に直接代入することはできない
Color b = (Color)4; // OK. 列挙型にキャストすれば代入できる
Color c = (Color)100; // エラーにはならないが未定義の動作(やってはいけない)
```

キャストを使うと、列挙子として割り当てていない値を代入できてしまうことに注意してください。<br>
C++では、これは「未定義の動作」とされています。

>未定義の動作とは「結果の予測ができない」という意味です。やってはいけません。


### 1.3 列挙型の比較

列挙型の変数や列挙子の比較は、列挙子の値同士の比較になります。

**コード**

```cpp
enum class Type {
  Dog,
  Lizard,
  Butterfly = -1,
};

int main() {
  Type a = Type::Lizard;

  if (a == Type::Dog) {
    cout << "犬" << endl;
  }
  else if (a != Type::Butterfly) {
    cout << "蝶ではない" << endl;
  }
  else {
    cout << "トカゲ" << endl;
  }

  if (Type::Butterfly < a) {
    cout << (int)a << "は蝶より大きい" << endl;
  }
}
```

**実行結果**

```txt
蝶ではない
トカゲ
1は蝶より大きい
```


### 1.4 列挙型の使いかた

先の例に出てきた`Color`列挙型のメンバは、色を識別するための何らかの値です。<br>
そして、`Red`の値が`1`だろうと`100`だろうと、「`Blue`や`Green`とは異なっている」と判定できれば十分です。

このように、列挙型は

>互いに異なるものとして扱いたいが、数値でも文字列でもあらわしにくいデータ

を扱いたい場合に使われます。

別の例として「ゲームキャラクターの行動状態」を考えてみましょう。<br>
このゲームキャラクターには次の4つの行動状態があるとします。

* 待機
* プレイヤーを攻撃
* プレイヤーを捜索
* 死亡

これらの状態は、単に現在の行動状態を示すためのもので、互いに区別できればどんな値でも構いません。<br>
このようなデータは列挙型に向いています。

```cpp
enum class ActionState {
  Idle,   // (アイドル)待機
  Attack, // (アタック)プレイヤーを攻撃
  Search, // (サーチ)プレイヤーを捜索
  Dead,   // (デッド)死亡
};
```

行動状態ということで、列挙型の名前は`ActionState`(アクション・ステート)としました。

実際のプログラムでは、ゲームキャラクターのクラスに`ActionState`型のメンバ変数を持たせます。<br>
そして、メンバ変数の値に応じて、キャラクターの動作プログラムを切り替えます。<br>
これは、おおよそ次のようなクラスになるでしょう。

```cpp
struct GameCharacter {
public:
  void Update() {
    switch (as) {
    case ActionState::Idle:
      // 待機を処理するプログラム
      if (プレイヤーを発見した) {
        as = ActionState::Attack;
      }
      break;

    case ActionState::Attack:
      // プレイヤーを攻撃するプログラム
      if (プレイヤーを見失った) {
        as = ActionState::Search;
      }
      break;

    case ActionState::Search:
      // プレイヤーを捜索するプログラム
      if (プレイヤーを見つけた) {
        as = ActionState::Attack;
      } else if (一定時間見つけられなかった) {
        as = ActionState::Idle;
      }
      break;

    case ActionState::Dead:
      // 死亡(何もしない)
      break;
    }
  }

private:
  ActionState as = ActionState::Idle;
};
```


#### 列挙型の利点

列挙型の利点は「存在しない値は設定できない」ことです(キャストを使わない限り)。

実のところ、列挙型の機能は、`int`や`string`などの数値型や文字列型を使っても実現できます。<br>
例えば、待機、攻撃、探索、死亡という4つの状態に`0`～`3`の数値を割り当てたり、文字列で`"Idle"`, "`Attack"`, `"Search"`, `"Dead"`などと表現することもできるでしょう。

問題は、数値型や文字列は、簡単に間違った値を指定できてしまうことです。<br>
例えば、整数の場合、状態をあらわす変数に、誤って`4`を代入してしまうかもしれません。<br>
文字列の場合、大文字と小文字を間違えて`"idle"`を設定してしまうかもしれません。

このような間違いがあっても、プログラムのビルドは成功してしまいます。そして、論理エラーが発生します。<br>
しかし、列挙型の場合は、存在しない値を指定しようとするとコンパイルエラーになります。間違いがすぐに分かるのです。<br>
これは、列挙型の最大の利点です。


### 1.5 C言語形式の列挙型

C言語形式の列挙型(以後は「Cの列挙型」と書きます)は、`enum`キーワードを使って次のように書きます。<br>
この形式では`class`キーワードは書きません。

```cpp
enum Suit {
  Club,
  Diamond,
  Heart,
  Spade,
};
```

Cの列挙型は、次の点でC++形式の列挙型とは挙動が異なります。

* `型名::`なしで列挙子を書ける
* キャストなしで整数型へ変換できる(整数型からの変換はキャストが必要)

```cpp
Suit a = Diamond;    // OK. C形式では`型名::`は不要
Suit b = Suit::Club: // OK. `型名::`を書くこともできる

int c = Space;       // OK. 数値型への変換はキャスト不要

//Suit d = 2;        // エラー. 数値数からの変換にはキャストが必要
Suit e = (Suit)2;    // OK. キャストすれば変換できる
```

Cの列挙型では、`型名::`を書かなくて済む代わりに、すべての列挙子に異なる名前を付ける必要があります。<br>
例えば、次のようなプログラムはエラーになります。

```cpp
enum COLOR {
  RED,
  GREEN,
  BLUE
};

enum SIGNAL {
  RED,   // エラー. REDという名前は既に使われている
  GREEN, // エラー. GREENという名前は既に使われている
  YELLOW
};
```

この制限を回避するため、Cの列挙型では「列挙子の名前に型名を含む」というルールが採用されていることがあります。

```cpp
enum COLOR {
  COLOR_RED,
  COLOR_GREEN,
  COLOR_BLUE
};
enum SIGNAL {
  SIGNAL_RED,   // OK. COLOR_RED と SIGNAL_RED は名前が異なる
  SIGNAL_GREEN, // OK. COLOR_GREEN と SIGNAL_GREEN は名前が異なる
  SIGNAL_YELLOW
};
```

さらに、Cの列挙型では暗黙の型変換が行われる、という問題があります。<br>
暗黙の型変換によって、数値が必要な部分に誤って列挙子を書いてしまってもエラーになりません。

例えば、次のプログラムはエラーになりません。

**コード**

```cpp
#include <iostream>
using namespace std;

enum COLOR {
  COLOR_RED,
  COLOR_GREEN,
  COLOR_BLUE,
};

double pow2(double a) { return a * a; }

int main() {
  int a = pow2(COLOR_BLUE); // 青色を2乗する？？？
  cout << a << endl;        // 4が出力される
}
```

**実行結果**

```txt
4
```

もし`COLOR`型が`enum class`で定義されていたら、`pow2(COLOR_BLUE)`の部分でコンパイルエラーになったでしょう。<br>
このように、Cの列挙型はバグの原因になりやすいのです。

C++11で`enum class`が追加されたのは、Cの列挙型が持つこれらの問題を解決するためです。<br>
新しいプログラムを書くときは、より安全な`enum clas`を使うべきです。

>**【Cの列挙型で大文字が使われる理由】**<br>
>これは、C言語の初期の教本において使われていた、「定数の名前はすべて大文字にする」という慣習の名残です。<br>
>そして、Cの列挙型は、伝統的に「定数」を定義する用途で使われています。<br>
>この合せ技により、列挙型や列挙子の名前は、すべて大文字になっていることが多いのです。
>
>対して、C++では、定数は`const`や`constexpr`キーワードを使って定義します。<br>
>そのため、列挙型を定数として使うことはほとんどなく、大文字小文字混じりの名前を付けることが多いです。


## 2 共用体


### 2.1 共用体を定義する

共用体は「メモリの節約」を目的とした特殊なクラスで、

&emsp;**同じメモリアドレスを複数のメンバ変数で共有し、一度にひとつのメンバ変数だけを有効にできる**

という機能を持っています。

>2025年現在、この目的にはC++17で追加された`variant`(バリアント)型が使われます(この型については後で説明します)。<br>
>しかし、C言語や、C++言語でもC++14以前のバージョンでは、共用体は現役で使われています。<br>
>また、`variant`は便利な機能の代償として、共用体より処理が遅くなる場合があります。<br>
>そのため、共用体について最低限の知識は持っておくとよいでしょう。

共用体を定義するには`union`(ユニオン)キーワードを使って、次のように書きます。

```cpp
union A {
  double d;
  int i;
};
```

ここで`A`は共用体の型名です。名前付けのルールは変数、構造体、関数などと同じです。

* 使える文字は英大文字、英小文字、数字、アンダーバーだけ
* 先頭1文字は数字以外

共用体のサイズは、「最も大きいメンバのサイズ」になります。<br>
例えばGoogle Colab環境では、`double`のサイズが`8`バイト、`int`のサイズが`4`バイトです。<br>
そのため、上記の共用体`A`のサイズは`8`バイトになります。

&emsp; $
union \space A \space \{ \\
\begin{array}{|c|c|c|} \hline
\begin{array}{c}double \space d; \\ int \space i; \end{array} & 8バイト　　 \\ \hline
\end{array} = 8バイト \\
\};
$

もし`A`が構造体だった場合、`A`型のサイズはすべてのメンバのサイズを足したものになります。<br>
つまり、構造体`A`のサイズは`12`バイトです。

&emsp; $
struct \space A \space \{ \\
\begin{array}{|c|c|c|} \hline
double \space d; & 8バイト　　 \\ \hline
int \space i;    & 4バイト　　 \\ \hline
\end{array} = 12バイト \\
\};
$

このように、共用体を使うとメモリを節約できることが分かります。<br>
ただし「同時にひとつのメンバしか使えない」という条件が付きます。


### 2.2 共用体のメンバを有効にする

共用体型の変数を宣言した時点では、どのメンバも無効になっています。

特定のメンバに値を代入すると、代入先のメンバだけが有効になります。<br>
その後、別メンバに値を代入すると、それまで有効だったメンバが無効化され、新しく代入されたメンバが有効になります。<br>
無効なメンバを読み出そうとすると、エラーにはなりませんが、大抵はおかしな値が返されます。

次のプログラムは、共用体のメンバの有効・無効を切り替える例です。

```cpp
union A {
  double d;
  int i;
};

int main() {
  A a;                    // 宣言の時点では d も i も無効
  // cout << a.i << endl; // NG. i は無効
  // cout << a.d << endl; // NG. d も無効

  a.d = 3.14;             // d が有効になる
  cout << a.d << endl;    // OK. d は最後に有効になったメンバ
  // cout << a.i << endl; // NG. この時点で i は無効

  a.i = 10;               // d は無効になり、i が有効になる
  // cout << a.d << endl; // NG. この時点で d は無効
  cout << a.i << endl;    // OK. i は最後に有効になったメンバ
}
```

この例で分かるように、適切に共用体を使うには、「今どのメンバが有効になっているか」を常に考えなくてはなりません。


### 2.3 共用体のメンバにコンストラクタ・デストラクタがある場合

コンストラクタやデストラクタを持つ型をメンバにする場合、コンストラクタとデストラクタを手動で呼び出す必要があります。<br>
次のプログラムは、コンストラクタとデストラクタを持つ型を、共用体のメンバとして使う例です。

```cpp
union A {
  int i;
  string s;
};

int main() {
  A a;

  a.i = 1; // OK. 基本型は何もしなくてよい

  // a.s = "abc";          // NG. string はコンストラクタを持っている
  new(&a.s) string("abc"); // OK. 特殊な new 構文でコンストラクタを呼び出す
  cout << a.s << endl;     // OK. s は有効になっている

  // a.i = 2;              // NG. この時点で有効な s はデストラクタを持っている
  a.s.~basic_string();     // OK. 有効な s に対して、手動で string のデストラクタを呼び出す. s は無効になる
  a.i = 2;                 // OK. s も i も無効になっている
}
```

このように、コンストラクタやデストラクタを持つ型を共用体のメンバにするのは、面倒な手順が必要になります。<br>
そして、手順を間違えると実行時エラーや論理エラーが起こります。

>慣れないうちは、共用体のメンバには`int`や`char`配列のような基本型のみを使うことをおすすめします。


### 2.4 共用体の制限



#### 継承の制限

共用体は、構造体と同じように、アクセス制御をしたり、メンバ関数を定義することができます。<br>
ただし、以下の機能は使えません。

* 継承を使えない(基底クラスを持てず、基底クラスにすることもできない)
* 参照型をメンバにできない

これらの機能が制限されているのは、共用体の「いつでも有効なメンバを変更できる」性質と相性が悪いためです。


#### 初期値の制限

共用体にも初期値を指定できます。ただし、指定できるのはメンバのうち一つだけです。

```cpp
// OK. メンバaだけ初期値を指定している
union A {
  int x;
  char a = 'S';
};

// エラー. 2つ以上のメンバに初期値を指定している
union B {
  int y = 123;
  char b = 'T';
};
```


### 2.5 共用体と列挙型を組み合わせる

共用体の有効なメンバを追跡する方法として、列挙型が使われることがあります。<br>
次のプログラムは、列挙型と共用体を組み合わせて使う例です。

**コード**

```cpp
struct Sample
{
  void SetDouble(double d) {
    am = ActiveMember::Double;
    u.d = d;
  }

  void SetInt(int i) {
    am = ActiveMember::Int;
    u.i = i;
  }

  // 有効なメンバをあらわす列挙型
  enum class ActiveMember {
    Double, // doubleが有効
    Int,    // intが有効
  } am = ActiveMember::Double;

  // 共用体
  union U {
    double d = 0.0;
    int i;
  } u;
};

int main() {
  Sample s;
  s.SetInt(123);

  // 列挙子を調べて、有効なメンバを選ぶ
  if (s.am == Sample::ActiveMember::Double) {
    cout << "double: " << s.u.d << endl;
  } else if (s.am == Sample::ActiveMember::Int) {
    cout << "int: " << s.u.i << endl;
  }
}
```

**実行結果**

```txt
int: 123
```

このプログラムでは、値の設定を関数にまとめています。<br>
関数を使うことで、有効なメンバと列挙子の変更が間違いなく行われることを保証できます。


### 2.6 無名共用体(むめい・きょうようたい)

共用体は型名を省略できます。型名を省略した共用体は「無名共用体」と呼ばれます。

あるスコープに無名共用体を書くと、そのスコープに名前のない共用体変数が宣言されます。<br>
そして、そのスコープ内では、名前のない共用体変数のメンバを直接読み書きできます。<br>

>これは無名共用体専用の特別ルールです。

次のプログラムは無名共用体の例です。

```cpp
#include <iostream>
using namespace std;

int main() {
  union { // ← 型名を書かない
    // main スコープの中では以下のメンバを直接読み書きできる
    int i;
    double d;
  };

  // 無名共用体のメンバ変数 i を操作
  i = 57;               // 無名共用体のメンバ i が有効になる
  cout << i << endl;    // OK. i は有効なメンバ
  // cout << d << endl; // NG. d は無効なメンバ

  // 無名共用体のメンバ変数 d を操作
  d = 3.14;             // 無名共用体のメンバ d が有効になる
  // cout << i << endl; // NG. i は無効なメンバ
  cout << d << endl;    // OK. d は有効なメンバ
}
```

無名共用体は、構造体のメンバとして使うと便利です。

**コード**

```cpp
struct Sample
{
  void SetDouble(double a) {
    am = ActiveMember::Double;
    d = a;
  }

  void SetInt(int a) {
    am = ActiveMember::Int;
    i = a;
  }

  enum class ActiveMember {
    Double, // doubleが有効
    Int,    // intが有効
  } am = ActiveMember::Double;

  // 無名共用体
  union {
    double d = 0.0;
    int i;
  };
};

int main() {
  Sample s;
  s.SetDouble(3.14);
  cout << s.d << endl; // s は無名共用体のメンバ d を直接読み書きできる

  s.SetInt(123);
  cout << s.i << endl; // s は無名共用体のメンバ i を直接読み書きできる
```

2.5節の`Sample`構造体では共用体に`u`という名前をつけていたため、メンバを参照するには`s.u.d`のように書く必要がありました。<br>
共用体を無名共用体に変えることで、名前`u`をなくして`s.d`のように短く書くことができます。


## 3 variant(バリアント)型


### 3.1 variant型の基本

`variant`(バリアント)型はC++17で追加された「安全な共用体」で、次のような特徴を持っています。

* 共用体とほぼ同じ仕組みで、メモリを節約できる
* 現在有効なメンバを自動的に管理する(プログラマが管理する必要がない)
* 必要に応じて自動的にコンストラクタとデストラクタを呼び出す<br>
  (共用体のように、手動で状態を管理する必要がない)
* 基底クラスにできる
* メンバにC配列を指定できない(`array`クラスで代用は可能)

共用体では、有効なメンバをプログラマが管理しなくてはなりませんでした。<br>
しかし、`variant`型は、有効なメンバを自動的に管理します。そのため、管理ミスによるバグは起きません。<br>
また、コンストラクタ・デストラクタを必要に応じて自動的に呼び出すため、それらの管理も不要になります。

`variant`型は、次のように定義されています。

```cpp
template <typename... Types>
class variant;
```

`variant`型では、メンバにしたい型をテンプレート引数として指定します。<br>
`typename...`という記述は、1つ以上のテンプレート引数をいくつでも指定できることを意味しています。

`variant`型を使うには`<variant>`ヘッダをインクルードします。<br>
次のプログラムは、`variant`型の変数を宣言する例です。

```cpp
#include <string>
#include <variant> // variant型を使う場合にインクルード
using namespace std;

int main() {
  // double, int, string の3つのうち、いずれかひとつだけを有効にできる変数を宣言する
  // 共用体と違って、最初のメンバ(この例ではdouble)が有効になる
  variant<double, int, string> a;

  // 初期値を指定すると、初期値と最もよく一致するメンバが有効になる  
  variant<double, int, string> b = "abc"; // string を有効化
  variant<double, int, string> c = 1;     // int を有効化
  variant<double, int, string> d = 1.0;   // double を有効化
}
```


### 3.2 variantから値を読み取る

`variant`型の変数`a`から値を読み取るには、`get`(ゲット)関数、または`get_if`(ゲット・イフ)関数を使います。<br>

>**書式**
>
>```cpp
>template<typename T>
>T& get(variant& a);
>```
>
>**テンプレート引数**
>
>* T&emsp;取得したいメンバの型
>
>**引数**
>
>* a&emsp;バリアント型の変数の参照
>
>**戻り値**
>
>Tが有効なメンバの場合、メンバの値を返す。<br>
>無効なメンバの場合は実行時エラーになる。

>**書式**
>
>```cpp
>template<typename T>
>T* get_if(variant* a);
>```
>
>**テンプレート引数**
>
>* T&emsp;取得したいメンバの型
>
>**引数**
>
>* a&emsp;バリアント型の変数のアドレス
>
>**戻り値**
>
>Tが有効なメンバの場合、メンバのアドレスを返す。<br>
>無効なメンバの場合は`nullptr`を返す。

`get`関数は、有効な型が分かっている場合に使います。テンプレート引数Tに無効な型を指定してしまうと、実行時エラーになります。<br>
`get_if`関数は、有効な型が分からない場合に使います。テンプレート引数に無効な形を指定した場合、`nullptr`が返されます。

次のプログラムは、`get`関数と`get_if`関数を使った例です。

```cpp
#include <iostream>
#include <variant>
#include <string>
using namespace std;

int main() {
  variant<double, int, string> a = "test"; // string が有効になる

  cout << get<string>(a) << endl;    // OK. string は最後に有効になったメンバ
  // cout << get<int>(a) << endl;    // エラー. intは無効

  a = 1;                             // string を無効にして int を有効にする
  // cout << get<string>(a) << endl; // エラー. stringは無効
  cout << get<int>(a) << endl;       // OK. int は最後に有効になったメンバ

  // double型の値の有無を調べる
  // double が有効だった場合は double のアドレスが返される
  // double が無効だった場合は nullptr が返される
  double* p = get_if<double>(&a); // アドレスを渡す点に注意
  if (p) {
    cout << *p << endl; // OK. double が無効ならこの行は実行されない
  }

  int* q = get_if<int>(&a); // アドレスを渡す点に注意
  if (q) {
    cout << *q << endl; // OK. int が無効ならこの行は実行されない
  }
}
```

**実行結果**

```txt
test
1
1
```


### 3.3 有効なメンバを調べる

`variant`の有効なメンバを調べるために、次の2つの機能が用意されています。

* `index`(インデックス)メンバ関数
* `holds_alternative`(ホールズ・オルタナティブ)関数

>**書式**
>
>```cpp
>size_t variant::index() const;
>```
>
>**戻り値**
>
>バリアント型を宣言する時に指定したテンプレート引数のうち、何番目のメンバが有効になっているかを返す。

>**書式**
>
>```cpp
>template<typename T>
>bool holds_alternative(const variant& a);
>```
>
>**テンプレート引数**
>
>* T&emsp;調べたいメンバの型
>
>**引数**
>
>* a&emsp;バリアント型の変数の参照
>
>**戻り値**
>
>Tが有効なメンバの場合は`true`を返す。<br>
>無効なメンバの場合は`false`を返す。

次のプログラムは、`index`と`holds_alternative`関数を使った例です。

```cpp
#include <iostream>
#include <string>
#include <variant>
using namespace std;

int main() {
  variant<string, double, int> a; // 0番目の string が有効になる

  cout << a.index() << endl; // 0が出力される
  cout << holds_alternative<string>(a) << ' '; // 1が出力される
  cout << holds_alternative<double>(a) << ' '; // 0が出力される
  cout << holds_alternative<int>(a) << endl;   // 0が出力される

  a = 365; // 2番めの int が有効になる
  cout << a.index() << endl; // 2が出力される
  cout << holds_alternative<string>(a) << ' '; // 0が出力される
  cout << holds_alternative<double>(a) << ' '; // 0が出力される
  cout << holds_alternative<int>(a) << endl;   // 1が出力される

  a = 1.41421356; // 1番目の double が有効になる
  cout << a.index() << endl; // 1が出力される
  cout << holds_alternative<string>(a) << ' '; // 0が出力される
  cout << holds_alternative<double>(a) << ' '; // 1が出力される
  cout << holds_alternative<int>(a) << endl;   // 0が出力される
}
```

**実行結果**

```txt
0
1 0 0
2
0 0 1
1
0 1 0
```

`index`メンバ関数は`switch`文と組み合わせて使うと便利です。<br>
`holds_alternative`は、基本的に`get`関数と組み合わせて使うのですが、大抵は`get_if`のほうが便利なので、あまり使われません。


### 3.4 usingエイリアス

`variant`型はテンプレート引数が多くなりがちなので、書くのが大変です。<br>
この面倒を減らすために「`using`エイリアスを使って別名を付ける」ということが行われます。

```cpp
#include <iostream>
#include <variant>
#include <string>
using namespace std;

using ParameterVariant = variant<double, int, string>;

int main() {
  ParameterVariant a; // variant<double, int, string> a; と同じ意味

  a = "abc";
  cout << get<string>(a) << endl;
}
```

このように別名を定義することで、プログラムを書きやすくなります。<br>
例えば、`variant<int, double, string>`のように順番を間違えてしまうことがなくなります。

**コード**

```cpp
  variant<double, int, string> a = 3.14;

  ︙

  variant<int, double, string> b = 3.14; // 順番を間違えた

  if (a == b) { // エラー. 型が違う
    cout << "同じ" << endl;
  } else {
    cout << "違う" << endl;
  }
}
```

また、`variant<double, int, string>`では抽象的すぎて、あらわしたいデータの種類がはっきり伝わりません。<br>
しかし`ParameterVariant`(パラメータ・バリアント)という名前は「何かのパラメータをあらわすんだな」という連想がしやすいです。

>**【別名を活用しよう】**<br>
>別名を付けると、エラーを起こしにくくなったり、プログラムを理解しやすくなります。


### 3.4 variantはいつ使う？

基本的に、`variant`型は「メモリを節約する」ことが目的です。そのため、どうしても`variant`型が必要、という場面は少ないです。

メモリの節約以外で`variant`型が役に立つのは、

&emsp;**実行してみるまで、データの種類が決められない**

ような場合です。

例えば、読み込んだデータファイルに書いてある内容によって、マップに配置する敵、障害物、イベント等を選ぶ場合が考えられます。<br>
これは次のような`variant`型で表現できます。

```cpp
struct Enemy {};    // 敵
struct Obstacle {}; // 障害物
struct Event {};    // イベント

variant<Enemy, Obstacle, Event> a;
```

なお、この例は「クラスの継承」とポインタを使っても同じことができます。<br>
例えば、次のようにします。

```cpp
struct MapObject {};            // マップに配置する物体の基底クラス

struct Enemy : MapObject {};    // 敵
struct Obstacle : MapObject {}; // 障害物
struct Event : MapObject {};    // イベント

shared_ptr<MapObject> a;
```

ただし、継承を使う方法では、事前に基底クラスを定義するなど、継承を使うように設計する必要があります。<br>
また、「動的メモリ管理」を使うため、メモリ管理が多少面倒になります。

対して、`variant`型は、継承関係にない型を組み合わせられます。<br>
また、`variant`型は「静的メモリ管理」や「自動メモリ管理」でも使えます。もちろん「動的メモリ管理」でもOKです。

これらの違いから、`variant`型と継承(または他の手段)のどちらを使うべきかは、状況によって異なります。<br>
仮想関数が必要などの理由で、既に継承を使った設計をしている場合は継承を選ぶべきでしょう。<br>
しかし、仮想関数も継承も不要だとか、後から無関係な型を組み合わせる必要が生じた場合は、`variant`型が役に立ちます。


## 4 optional(オプショナル)型


### 4.1 optional型の基本

データの読み込みに失敗したり、探しているデータが見つからなかったなど、「データがない」ことをあらわしたい場合があります。<br>
ポインタ型の場合、これは`nullptr`を使うことで実現できます。しかし、ポインタではない型に`nullptr`は使えません。

そこで考案されたのが`optional`型です。<br>
`optional`型を使うと、どんな型に対しても「値が無効な状態」と「値が有効な状態」の2つの状態をあらわせます。

`optional`型は、次のように定義されています。

```cpp
template <typename T>
class optional;
```

有効な値を取り出すには`value`(バリュー)メンバ関数を使います。

>**書式**
>
>```cpp
>T& optional::value();
>```
>
>**戻り値**
>
>有効な場合は格納されている値を返す。<br>
>無効な場合は実行時エラー。<br>

また、`optional`型の無効な値をあらわすために、`nullopt`(ヌル・オプト)という特殊な定数が用意されています。

`optional`型を使うには`<optional>`ヘッダをインクルードします。<br>
次のプログラムは、`optional`型の変数を宣言する例です。

**コード**

```cpp
#include <iostream>
#include <optional> // optional型を使う場合にインクルード
using namespace std;

int main() {
  optional<int> a; // 無効な状態
  optional<int> b = nullopt; // 無効な状態
  optional<int> c = 0; // 有効な状態

  // cout << a.value() << endl; // エラー. aは無効な状態
  // cout << b.value() << endl; // エラー. bは無効な状態
  cout << c.value() << endl;    // OK. 変数cは有効な値を持っている
}
```
**実行結果**

```txt
0
```


### 4.2 optionalの有効・無効を判定する

`optional`型の有効・無効を判定するには、`optional`型の変数を条件式に指定します。

**コード**

```cpp
#include <iostream>
#include <optional> // optional型を使う場合にインクルード
using namespace std;

optional<int> f(int x, int y) {
  if (y == 0) {
    return nullopt; // 無効を意味する nullopt を返す
  }
  return x / y; // 有効な値を返す
}

int main() {
  auto a = f(10, 3);

  if (a) {
    cout << a.value() << endl; // valueメンバ関数で値を取り出す
  } else {
    cout << "error" << endl;
  }

  auto b = f(7, 0);
  if (b) {
    cout << b.value() << endl; // valueメンバ関数で値を取り出す
  } else {
    cout << "error" << endl;
  }
}
```

**実行結果**

```txt
3
error
```




### 4.3 optionalはいつ使う？

基本的な考え方は「失敗があり得る関数の戻り値」として使うことです。

例えば前のプログラム例の関数`f`は、整数の除算を行います。しかし、もし除数`y`が`0`の場合は除算を実行できません。<br>
そこで、`optional`型を使って「関数の結果は無効(`y`が`0`なので計算できない)」という情報を返しています。

```cpp
optional<int> f(int x, int y) {
  if (y == 0) {
    return nullopt;
  }
  return x / y;
}
```

ただし、無効値を表現できるポインタ型では、わざわざ`optional`を使うまでもありません。<br>
また、単に成功と失敗を返したいだけなら`bool`型を使うべきです。

>**【詳細なエラー情報が必要な場合はvariant型を検討する】**<br>
>「失敗した」という単純な情報ではなく、より詳細なエラー情報を返したい場合は、`variant`型を使うとよいでしょう。


----

## 5 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１ ランク付け

試験の成績が10人分入力されます。<br>
以下のルールに従って`Low`(ロー), `Middle`(ミドル)、`High`(ハイ)に分類するプログラムを作成しなさい。

* 30点未満: `Low`に分類
* 30点以上かつ80点未満: `Middle`に分類
* 80点以上: `High`に分類

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. if文を使って、変数`a`が30より小さい場合、`ranks[i]`に列挙子`Rank::Low`を代入する
2. else if文を使って、変数`a`が80より小さい場合、`ranks[i]`に列挙子`Rank::Middle`を代入する
3. else句を使って、`ranks[i]`に列挙子`Rank::High`を代入する

</details><br>

**入力データ例（１）**

```txt
29 30 80 79 100 65 0 18 40 31
```

**出力例（１）**

```txt
Low: 3
Middle: 5
High: 2
```

**入力データ例（２）**

```txt
1 2 3 4 5 6 7 8 9 10
```

**出力例（２）**

```txt
Low: 10
Middle: 0
High: 0
```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <algorithm>
using namespace std;

// 成績ランク
enum class Rank {
  Low,    // 30点未満
  Middle, // 30点以上、80点未満
  High,   // 80点以上
};

int main() {
  vector<Rank> ranks(10);
  for (int i = 0; i < 10; i++) {
    int a;
    cin >> a;
    // --- ここから上は変更しない ---

    // この下に、得点に対応する列挙子を配列に代入するプログラムを書く


    // --- ここから下は変更しない ---
  }

  cout << "Low: " << count(ranks.begin(), ranks.end(), Rank::Low) << endl;
  cout << "Middle: " << count(ranks.begin(), ranks.end(), Rank::Middle) << endl;
  cout << "High: " << count(ranks.begin(), ranks.end(), Rank::High) << endl;
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、10人の得点を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "Low: 10\nMiddle: 0\nHigh: 0\nLow: 3\nMiddle: 5\nHigh: 2\nLow: 3\nMiddle: 3\nHigh: 4") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo -e "1 2 3 4 5 6 7 8 9 10" | ./practice_01a && echo -e "29 30 80 79 100 65 0 18 40 31" | ./practice_01a && echo -e "100 99 29 30 31 79 80 81 1 0" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <algorithm>
using namespace std;

// 成績ランク
enum class Rank {
  Low,    // 30点未満
  Middle, // 30点以上、80点未満
  High,   // 80点以上
};

int main() {
  vector<Rank> ranks(10);
  for (int i = 0; i < 10; i++) {
    int a;
    cin >> a;
    // --- ここから上は変更しない ---

    // この下に、得点に対応する列挙子を配列に代入するプログラムを書く
    if (a < 30) {
      ranks[i] = Rank::Low;
    } else if (a < 80) {
      ranks[i] = Rank::Middle;
    } else {
      ranks[i] = Rank::High;
    }

    // --- ここから下は変更しない ---
  }

  cout << "Low: " << count(ranks.begin(), ranks.end(), Rank::Low) << endl;
  cout << "Middle: " << count(ranks.begin(), ranks.end(), Rank::Middle) << endl;
  cout << "High: " << count(ranks.begin(), ranks.end(), Rank::High) << endl;
}

### ❓️問題２

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.
4.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

### ❓️問題３ 点と箱の共用体

`Shape`(シェイプ)型は、点と箱のどちらかをあらわすことができる共用体です。

N個の点または箱のデータが入力されます。<br>
データを`Shape`型の配列に保存し、入力されたのと逆順で出力するプログラムを作成しなさい。

点と箱のデータは次の形式で入力されます

* 点のデータ: `P X座標 Y座標`
* 箱のデータ: `B 左上X座標 左上Y座標 右下X座標 右下Y座標`

**入力データ形式**

```txt
データ数
点または箱のデータ1
点または箱のデータ2
︙
点または箱のデータN
```

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `int`型の変数`n`を宣言し、`cin`から`n`にデータ数を読み込む
2. `vector<Shape>`型の配列変数`shapes`(シェイプス)を宣言し、`n`個で初期化する
3. for文を使って、次の処理を`n`回繰り返す
    1. `Shape&`型の参照変数`s`を宣言し、`shapes[i]`で初期化する
    2. `cin`から`s.type`にデータの種類を読み込む
    3. if文を使って、`s.type`が文字`P`の場合、次の処理を実行する
        1. `cin`からメンバ変数`s.point.x`に、点のX座標を読み込む
        2. `cin`からメンバ変数`s.point.y`に、点のY座標を読み込む
    4. else if文を使って、`s.type`が文字`B`の場合、次の処理を実行する
        1. `cin`からメンバ変数`s.box.min.x`に、箱の左上X座標を読み込む
        2. `cin`からメンバ変数`s.box.min.y`に、箱の左上X座標を読み込む
        3. `cin`からメンバ変数`s.box.max.x`に、箱の右下X座標を読み込む
        4. `cin`からメンバ変数`s.box.max.y`に、箱の右下X座標を読み込む
4. `reverse`汎用アルゴリズムを使って、`shapes`配列を逆順にする
5. for文を使って、次の処理を`n`回繰り返す
    1. `const Shape&`型の参照変数`s`を宣言し、`shapes[i]`で初期化する
    2. if文を使って、`s.type`が文字`P`の場合、次の処理を実行する
        1. `cout`に文字`P`と、空白1文字を出力する
        2. `cout`にメンバ変数`s.point.x`と、空白1文字を出力する
        3. `cout`にメンバ変数`s.point.y`を出力し、改行する
    3. else if文を使って、`s.type`が文字`B`の場合、次の処理を実行する
        1. `cout`に文字`B`と、空白1文字を出力する
        2. `cout`にメンバ変数`s.box.min.x`と、空白1文字を出力する
        3. `cout`にメンバ変数`s.box.min.y`と、空白1文字を出力する
        4. `cout`にメンバ変数`s.box.max.x`と、空白1文字を出力する
        5. `cout`にメンバ変数`s.box.max.y`を出力し、改行する


</details><br>

**入力データ例（１）**

```txt
4
P 4 1
B 2 0 3 3
P 3 3
B 3 5 6 7
```

**出力例（１）**

```txt
B 3 5 6 7
P 3 3
B 2 0 3 3
P 4 1
```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <vector>
#include <algorithm>
using namespace std;

// 点の構造体
struct Point { int x, y; };

// 箱の構造体
struct Box { Point min, max; };

// 点と箱の共用体
struct Shape
{
  char type; // データの種類 `P`=点 `B`=箱

  union {
    Point point; // 点
    Box box;     // 箱
  };
};

int main() {
  // この下に、点と箱のデータを逆順で出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、点と箱のリストを入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "B 3 5 6 7\nP 3 3\nB 2 0 3 3\nP 4 1\nP 2 3\nB 4 5 6 7") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo -e "4 P 4 1 B 2 0 3 3 P 3 3 B 3 5 6 7" | ./practice_02a && echo -e "1 P 2 3" | ./practice_02a && echo -e "1 B 4 5 6 7" | ./practice_02a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <vector>
#include <algorithm>
using namespace std;

// 点の構造体
struct Point { int x, y; };

// 箱の構造体
struct Box { Point min, max; };

// 点と箱の共用体
struct Shape
{
  char type; // データの種類 `P`=点 `B`=箱

  union {
    Point point; // 点
    Box box;     // 箱
  };
};

int main() {
  // この下に、点と箱のデータを逆順で出力するプログラムを書く
  int n;
  cin >> n;

  vector<Shape> shapes(n);
  for (int i = 0; i < n; i++) {
    Shape& s = shapes[i];
    cin >> s.type;

    if (s.type == 'P') {
      cin >> s.point.x >> s.point.y;
    } else if (s.type == 'B') {
      cin >> s.box.min.x >> s.box.min.y >> s.box.max.x >> s.box.max.y;
    }
  }

  reverse(shapes.begin(), shapes.end());

  for (int i = 0; i < n; i++) {
    const Shape& s = shapes[i];
    if (s.type == 'P') {
      cout << "P " << s.point.x << " " << s.point.y << endl;
    }
    else if (s.type == 'B') {
      cout << "B " << s.box.min.x << " " << s.box.min.y << " " << s.box.max.x << " " << s.box.max.y << endl;
    }
  }
}

### ❓️問題４

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.
4.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

### ❓️問題５

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.
4.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

### ❓️問題６

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.
4.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

### ❓️問題７

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.
4.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

### ❓️問題８

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.
4.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a && echo -e "" | ./practice_01a) > /dev/null && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <vector>
#include <variant>
#include <optioanl>
using namespace std;

int main() {
  // この下に、するプログラムを書く

}